# Group Assignment QF627

*Group Members:*
* Anna Germaine Lim
* Peng Cheng
* Zenith Tay
* Gregory Tan

## Packages Used in This Workbook

In [34]:
## Data Download

import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime

## For Visualisation

## Useful Functions Used in this Sheet

In [35]:
def download_data (ticker: list[str] | str,
                   start_date: str | datetime, 
                   end_date: str | datetime) -> pd.DataFrame:
    data =\
    (
        yf.download(tickers = ticker,
                    start = start_date,
                    end = end_date)
    )

    return data

In [45]:
def generate_returns(price_data: pd.DataFrame,
                     price_col: 'str' = 'Close') -> pd.Series:
    
    returns_series = price_data[price_col].pct_change()
    cum_returns_series = (1 + returns_series).cumprod()

    tot_returns = float(cum_returns_series.iloc[-1])

    print('=== Summary of Returns === \n',
          f'Total Returns = {tot_returns: .2%}'
          )

    return tot_returns, returns_series, cum_returns_series

In [37]:
def mean_reversion_strategy(price_data: pd.DataFrame):
    return None

## Download Dataset

In [46]:
spy_data =\
(
    download_data('SPY',
                  start_date = '2006-11-01',
                  end_date = '2025-11-12')
    .droplevel(level = 1,
               axis = 1)
)

C:\Users\TanFamily4\AppData\Local\Temp\ipykernel_16024\848347459.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download(tickers = ticker,
[*********************100%***********************]  1 of 1 completed


In [47]:
generate_returns(spy_data)

=== Summary of Returns === 
 Total Returns =  711.65%


(7.116540777494998,
 Date
 2006-11-01         NaN
 2006-11-02   -0.000585
 2006-11-03   -0.001755
 2006-11-06    0.011279
 2006-11-07    0.003838
                 ...   
 2025-11-05    0.003465
 2025-11-06   -0.010729
 2025-11-07    0.000985
 2025-11-10    0.015604
 2025-11-11    0.002289
 Name: Close, Length: 4787, dtype: float64,
 Date
 2006-11-01         NaN
 2006-11-02    0.999415
 2006-11-03    0.997662
 2006-11-06    1.008914
 2006-11-07    1.012787
                 ...   
 2025-11-05    7.060067
 2025-11-06    6.984317
 2025-11-07    6.991193
 2025-11-10    7.100286
 2025-11-11    7.116541
 Name: Close, Length: 4787, dtype: float64)

In [40]:
spy_data

Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2006-11-01,95.973595,96.990411,95.875420,96.927300,83005600
2006-11-02,95.917496,96.078781,95.622970,95.728154,60693100
2006-11-03,95.749191,96.345261,95.104039,96.261114,71346400
2006-11-06,96.829147,96.969396,96.036727,96.043746,63303300
2006-11-07,97.200813,97.460274,96.773048,96.913297,63318900
...,...,...,...,...,...
2025-11-05,677.580017,680.859985,674.169983,674.979980,74402400
2025-11-06,670.309998,677.380005,668.719971,676.469971,85035300
